In [48]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
from tqdm import tqdm
import scipy
import ta
import talib

In [49]:
#신세계푸드 031440 / 신세계건설 034300 / 신세계 I&C 035510
# 광주신세계 037710 / (주식회사)신세계 004170 / (주식회사)이마트 139480 / 현대건설 000720 / CJ제일제당 097950
codes=['031440','034300','035510','004170','139480','000720','097950']

In [50]:
glob.glob('./data/종목데이터/df*.csv')[0][-13:-4]

'df_000720'

In [51]:
# 데이터 불러오기
for file in glob.glob('./data/종목데이터/df_*.csv'):
    globals()['{}'.format(file[-13:-4])] = pd.read_csv('{}'.format(file))

kospi = pd.read_csv('./data/kospi.csv')
ex_usd = pd.read_csv('./data/환율데이터/exchange_rate_USD.csv')
ex_cny = pd.read_csv('./data/환율데이터/exchange_rate_CNY.csv')

In [52]:
#신세계푸드 031440 /(주식회사)신세계 004170/(주식회사)이마트 139480
main_codes = ['031440','004170','139480']

In [53]:
sub_codes = list(set(codes) - set(main_codes))

In [54]:
sub_codes

['000720', '097950', '034300', '035510']

In [55]:
for code in main_codes:
    print('df_{}'.format(code))

df_031440
df_004170
df_139480


In [56]:
for code in sub_codes:
    print('df_{}'.format(code))

df_000720
df_097950
df_034300
df_035510


In [57]:
# 추가 컬럼 
def add_col(df):
    
    df.rename(columns={'qty':'volume'},inplace=True)

    # 등락률 계산
    # 직전 종가 대비
    df["close yesterday"] = df["close"].shift(1)
    df['volume yesterday'] = df['volume'].shift(1)

    df["ratio_close"] = ((df["close"] - df["close yesterday"]) / df["close yesterday"])*100
    df["ratio_high_close"] = ((df["high"] - df["close yesterday"]) / df["close yesterday"])*100
    df["ratio_low_close"] = ((df["low"] - df["close yesterday"]) / df["close yesterday"])*100
    df["ratio_open_close"] = (df["open"] - df["close yesterday"]) / df["close yesterday"]*100
    
    df['ratio_volume'] = ((df['volume']) / df['volume yesterday'])*100
    df['ratio_volume'].fillna(0, inplace=True)
    df['ratio_volume'] = df['ratio_volume'].replace([np.inf,-np.inf],0)
 
    # 이동평균선 (MA20)
    df['MA20'] = talib.SMA(df['close'], 20)
    df['ratio_MA20_close'] = (df['close'] /  talib.SMA(df['close'], 20)) *100
    df['ratio_MA20_volume'] = (df['volume'] /  talib.SMA(df['volume'], 20)) *100
    df['ratio_MA20_volume'].fillna(0, inplace=True)
    df['ratio_MA20_volume'] = df['ratio_MA20_volume'].replace([np.inf,-np.inf],0)

    return df

In [58]:
def ma_add_col(df,colname):

    df["close yesterday"] = df[colname].shift(1)
    df["change"] = ((df[colname] - df["close yesterday"]) / df["close yesterday"])*100

    # 이동평균선 (MA20)
    df['MA20'] = talib.SMA(df[colname], 20)
    df['ratio_MA20_close'] = (df[colname] / df['MA20']) *100

    return df

In [59]:
for code in codes:
    globals()['add_{}'.format(code)] = add_col(globals()['df_{}'.format(code)])

In [60]:
df_035510.iloc[120:,:]

,Datetime,open,high,low,close,volume,value,close yesterday,volume yesterday,ratio_close,ratio_high_close,ratio_low_close,ratio_open_close,ratio_volume,MA20,ratio_MA20_close,ratio_MA20_volume
120,2021-06-28 11:01:00,18750.0,18750.0,18750.0,18750.0,70,1310000,18700.0,0.0,0.267380,0.267380,0.267380,0.267380,0.000000,18765.0,99.920064,16.412661
121,2021-06-28 11:02:00,18750.0,18750.0,18700.0,18700.0,290,5420000,18750.0,70.0,-0.266667,0.000000,-0.266667,0.000000,414.285714,18762.5,99.666889,68.396226
122,2021-06-28 11:03:00,18700.0,18750.0,18700.0,18700.0,1000,18700000,18700.0,290.0,0.000000,0.267380,0.000000,0.000000,344.827586,18760.0,99.680171,232.288037
123,2021-06-28 11:04:00,18700.0,18750.0,18700.0,18750.0,270,5060000,18700.0,1000.0,0.267380,0.267380,0.000000,0.000000,27.000000,18760.0,99.946695,63.754427
124,2021-06-28 11:05:00,18750.0,18750.0,18700.0,18700.0,170,3180000,18750.0,270.0,-0.266667,0.000000,-0.266667,0.000000,62.962963,18757.5,99.693456,39.859320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194175,2023-07-20 15:16:00,14010.0,14010.0,14010.0,14010.0,37,518370,14010.0,7.0,0.000000,0.000000,0.000000,0.000000,528.571429,14017.5,99.946495,37.755102
194176,2023-07-20 15:17:00,14010.0,14010.0,14010.0,14010.0,109,1527090,14010.0,37.0,0.000000,0.000000,0.000000,0.000000,294.594595,14016.5,99.953626,108.188586
194177,2023-07-20 15:18:00,14010.0,14010.0,14010.0,14010.0,177,2479770,14010.0,109.0,0.000000,0.000000,0.000000,0.000000,162.385321,14015.5,99.960758,165.343298
194178,2023-07-20 15:19:00,14010.0,14010.0,14000.0,14010.0,210,2942100,14010.0,177.0,0.000000,0.000000,-0.071378,0.000000,118.644068,14015.0,99.964324,188.256387


In [61]:
# 직전가 컬럼 추가
for code in codes:
    last_price = globals()['add_{}'.format(code)]['close'].tolist()
    last_price.insert(0,0)
    last_price = last_price[:-1]

    globals()['add_{}'.format(code)]['last_price'] = last_price

In [63]:
# 대비 부호 추가하기
# 상승:1    보합:0   하락:-1 

for code in codes:
    sign = [0]
    for i in range(1,len(globals()['add_{}'.format(code)])):
            
        if globals()['add_{}'.format(code)]['last_price'][i] < globals()['add_{}'.format(code)]['close'][i]:
            sign.append(1)
        elif globals()['add_{}'.format(code)]['last_price'][i] == globals()['add_{}'.format(code)]['close'][i]:
            sign.append(0)
        elif globals()['add_{}'.format(code)]['last_price'][i] > globals()['add_{}'.format(code)]['close'][i]:
            sign.append(-1)

        i+=1
    globals()['add_{}'.format(code)]['sign'] = sign

In [64]:
# 서브 종목들 컬럼명 변경
for code in sub_codes:
    col_list=['Datetime']
    
    for col in globals()['add_{}'.format(code)].columns[1:]:
        
        col_name = 'D' + code +'_'+ col
        col_list.append(col_name)
    
    globals()['add_{}'.format(code)].columns = [col_list]

In [65]:
kospi = ma_add_col(kospi, '종가')

In [66]:
ex_cny = ma_add_col(ex_cny,'매매기준율')

In [67]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [68]:
ex_usd['매매기준율'] = ex_usd['매매기준율'].apply(lambda _ : locale.atof(_))

In [69]:
ex_usd= ma_add_col(ex_usd,'매매기준율')

In [70]:
kospi

,Datetime,고가,저가,종가,close yesterday,change,MA20,ratio_MA20_close
0,2021-06-28 09:01:00,3308.520020,3304.860107,3302.969971,NaN,NaN,NaN,NaN
1,2021-06-28 09:02:00,3307.860107,3302.389893,3302.909912,3302.969971,-0.001818,NaN,NaN
2,2021-06-28 09:03:00,3304.020020,3302.399902,3300.129883,3302.909912,-0.084169,NaN,NaN
3,2021-06-28 09:04:00,3302.820068,3300.129883,3298.739990,3300.129883,-0.042116,NaN,NaN
4,2021-06-28 09:05:00,3299.639893,3298.590088,3298.919922,3298.739990,0.005455,NaN,NaN
...,...,...,...,...,...,...,...,...
194175,2023-07-20 15:16:00,2605.949951,2604.090088,2605.719971,2605.760010,-0.001537,2605.145984,100.022033
194176,2023-07-20 15:17:00,2604.280029,2603.479980,2604.090088,2605.719971,-0.062550,2605.174988,99.958356
194177,2023-07-20 15:18:00,2604.010010,2602.989990,2603.939941,2604.090088,-0.005766,2605.207483,99.951346
194178,2023-07-20 15:19:00,2604.479980,2603.570068,2603.899902,2603.939941,-0.001538,2605.164978,99.951440


In [71]:
kospi = kospi[['Datetime','change','ratio_MA20_close']]
kospi.columns = [['Datetime','kospi_change','kospi_ratio_MA20_close']]

In [72]:
ex_usd = ex_usd[['Datetime','change','ratio_MA20_close']]
ex_usd.columns = [['Datetime','USD_change','USD_ratio_MA20_close']]
ex_cny = ex_cny[['Datetime','change','ratio_MA20_close']]
ex_cny.columns = [['Datetime','CNY_change','CNY_ratio_MA20_close']]

In [73]:
ex_usd

,Datetime,USD_change,USD_ratio_MA20_close
0,2021-06-28 09:01:00,NaN,NaN
1,2021-06-28 09:02:00,0.000000,NaN
2,2021-06-28 09:03:00,0.000000,NaN
3,2021-06-28 09:04:00,0.000000,NaN
4,2021-06-28 09:05:00,0.053196,NaN
...,...,...,...
194175,2023-07-20 15:16:00,0.023646,100.101758
194176,2023-07-20 15:17:00,0.031521,100.123044
194177,2023-07-20 15:18:00,0.039389,100.152212
194178,2023-07-20 15:19:00,0.000000,100.140760


In [74]:
ex_usd

,Datetime,USD_change,USD_ratio_MA20_close
0,2021-06-28 09:01:00,NaN,NaN
1,2021-06-28 09:02:00,0.000000,NaN
2,2021-06-28 09:03:00,0.000000,NaN
3,2021-06-28 09:04:00,0.000000,NaN
4,2021-06-28 09:05:00,0.053196,NaN
...,...,...,...
194175,2023-07-20 15:16:00,0.023646,100.101758
194176,2023-07-20 15:17:00,0.031521,100.123044
194177,2023-07-20 15:18:00,0.039389,100.152212
194178,2023-07-20 15:19:00,0.000000,100.140760


In [75]:
def add_col_exko(df):
    df[['kospi_change','kospi_ratio_MA20_close']] = kospi[['kospi_change','kospi_ratio_MA20_close']]
    df[['USD_change','USD_ratio_MA20_close']] = ex_usd[['USD_change','USD_ratio_MA20_close']]
    df[['CNY_change','CNY_ratio_MA20_close']] = ex_cny[['CNY_change','CNY_ratio_MA20_close']]

    return df

In [76]:
# 메인 종목 3개 환율, 코스피 추가
for code in main_codes:
    globals()['add_{}'.format(code)] = add_col_exko(globals()['add_{}'.format(code)])

In [77]:
add_031440.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194180 entries, 0 to 194179
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Datetime                194180 non-null  object 
 1   open                    194180 non-null  float64
 2   high                    194180 non-null  float64
 3   low                     194180 non-null  float64
 4   close                   194180 non-null  float64
 5   volume                  194180 non-null  int64  
 6   value                   194180 non-null  int64  
 7   close yesterday         194179 non-null  float64
 8   volume yesterday        194179 non-null  float64
 9   ratio_close             194179 non-null  float64
 10  ratio_high_close        194179 non-null  float64
 11  ratio_low_close         194179 non-null  float64
 12  ratio_open_close        194179 non-null  float64
 13  ratio_volume            194180 non-null  float64
 14  MA20                

In [78]:
# 데이터 저장
for code in codes:
    globals()['add_{}'.format(code)].to_csv('./data/add_{}.csv'.format(code),index=False)